In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gymnasium as gym
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions.categorical import Categorical

In [3]:
import sys
sys.argv = [""]

In [10]:
from cart_pole.utils import *
from cart_pole.agent import Agent
from torch.utils.tensorboard import SummaryWriter
from cart_pole.environment import *
import pdb

In [13]:
args = parse_args(seconds_since_epoch=1764177877)
writer = init_tracking(args)

In [59]:
class Storage(object):

    def __init__(self,
                 args,
                 envs):

        self.num_steps = args.num_steps
        self.num_envs = args.num_envs
        self.device = args.device

        self.reset(envs)

    def is_env_running(self):
        return (self.episode_info["env_state"] == "Running").values
    
    def store_state_info(self,
                         obs,
                         terminated,
                         truncated,
                         info):

        if self.step is None:
            self.step = 0
        else:
            self.step += 1

        assert self.step < self.num_steps, "Storage is full"

        is_env_running = self.is_env_running()
        self.obs[self.step, is_env_running, :] = obs[is_env_running, :]

        for env_idx in range(self.num_envs):

            if self.episode_info.iloc[env_idx]["env_state"] == "Running":

                if terminated[env_idx]:

                    self.episode_info.iloc[env_idx]["env_state"] =\
                        "Terminated"
                #-------------------------------------------------
                elif truncated[env_idx]:

                    self.episode_info.iloc[env_idx]["env_state"] =\
                        "Terminated"

        if all(key in info for key in ["episode", "_episode"]):

            select_row = info["_episode"]

            self.episode_info.loc[select_row, "cumulative_reward"] =\
                info["episode"]["r"][select_row]

            self.episode_info.loc[select_row, "episode_length"] =\
                info["episode"]["l"][select_row]

    def store_action_info(self,
                          values,
                          action,
                          logprob):
        pass

    def reset(self,
              envs):

        self.step = None

        shape2d = (self.num_steps, self.num_envs) 
        shape3d = shape2d + envs.single_observation_space.shape

        self.obs = torch.zeros(shape3d).to(self.device)
        self.actions = torch.zeros(shape3d).to(self.device)

        self.logprobs = torch.zeros(shape2d).to(self.device)
        self.rewards = torch.zeros(shape2d).to(self.device)
        self.values = torch.zeros(shape2d).to(self.device)

        self.episode_info =\
            pd.DataFrame([{"cumulative_reward": 0,
                           "episode_length": 0,
                           "env_state": "Running"}] *\
                         self.num_envs)

In [60]:
rng = init_rngs(args)
seeds = [int(rng.uniform(0, 1E9)) for _ in range(args.num_envs)]

envs = make_sync_vector_env(args)
storage = Storage(args, envs)
agent = Agent(envs).to(args.device)

optimizer = optim.Adam(agent.parameters(),
                       lr=args.learning_rate,
                       eps=1e-5)

next_obs = torch.Tensor(envs.reset(seed=seeds)[0]).to(args.device)
terminated = np.array([False] * args.num_envs)
truncated = np.array([False] * args.num_envs)
info = {}

num_updates = args.total_timesteps // args.batch_size
storage.reset(envs)

storage.store_state_info(next_obs,
                         terminated,
                         truncated,
                         info)

In [61]:
storage.episode_info

,cumulative_reward,episode_length,env_state
0,0,0,Running
1,0,0,Running
2,0,0,Running
3,0,0,Running


In [30]:




for _ in range(args.num_steps):

    storage.store_state_info(next_obs,
                             terminated,
                             truncated,
                             info)

    with torch.no_grad():

        actions, logprobs, _, values = agent.get_action_and_value(next_obs)

        storage.store_action_info(values.flatten(),
                                  action,
                                  logprob)

    next_obs, rewards, terminated, truncated, info =\
        envs.step(action.cpu().numpy())
    
    storage.store("rewards",
                  torch.tensor(reward).to(args.device).view(-1))

    next_obs = torch.Tensor(next_obs).to(args.device)

> /tmp/ipykernel_2604/3888838010.py(26)<module>()
     24         pdb.set_trace()
     25 
---> 26     with torch.no_grad():
     27         action, logprob, _, value = agent.get_action_and_value(next_obs)
     28         storage.store("values", value.flatten())



ipdb>  next_termined


*** NameError: name 'next_termined' is not defined


ipdb>  next_terminated


tensor([0., 1., 0., 0.], device='cuda:0')


...

ipdb>  storage.terminated[:29,:]


tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 1., 0., 0.]], device='cuda:0')


ipdb>  info


{'episode': {'r': array([ 0., 18.,  0.,  0.]), '_r': array([False,  True, False, False]), 'l': array([ 0, 18,  0,  0]), '_l': array([False,  True, False, False]), 't': array([ 0.      , 60.161961,  0.      ,  0.      ]), '_t': array([False,  True, False, False])}, '_episode': array([False,  True, False, False])}


ipdb>  q


In [27]:
next_obs

tensor([[-0.0056, -0.3969, -0.0460,  0.1331],
        [ 0.0143,  0.4369, -0.0547, -0.6539],
        [-0.0427, -0.5255,  0.0263,  0.4999],
        [-0.0428, -0.2076,  0.0210,  0.3240]], device='cuda:0')

In [25]:
storage.terminated.argmax(axis=0)

tensor([26,  9, 36, 40], device='cuda:0')